In [107]:
import numpy as np
from PIL import Image as im
from sys import exit

In [108]:
def pgm_to_matrix(image):
    matrix = list(image.getdata())
    width, height = image.size
    return [matrix[i:i+width] for i in range(0, len(matrix), width)]

In [109]:
def matrix_to_pgm(matrix):
    image_array = np.array(matrix, dtype=np.uint8)
    return im.fromarray(image_array)

In [110]:
def print_arr(arr):
    rows = len(arr)

    for y in range(rows):
        print(arr[y], "\n")

In [121]:
def reduce_matrix(matrix, scale):
    rows = len(matrix)
    cols = len(matrix[0])

    rrows = rows // scale
    rcols = cols // scale

    rmatrix = [[0 for _ in range(rcols)] for _ in range(rrows)]

    scale_sqr = scale**2

    for y in range(rrows):
        my = y * scale
        for x in range(rcols):
            mx = x * scale
            for i in range(scale):
                for j in range(scale):
                    rmatrix[x][y] += matrix[mx + i][my + j]
            rmatrix[x][y] /= scale_sqr

    return rmatrix

In [130]:
def extend_matrix_1(matrix, scale):
    rows = len(matrix)
    cols = len(matrix[0])

    rrows = rows * scale
    rcols = cols * scale

    rmatrix = [[0 for _ in range(rcols)] for _ in range(rrows)]

    # nearest neighbor
    for y in range(rrows):
        ry = y // scale
        for x in range(rcols):
            rmatrix[x][y] = matrix[x // scale][ry]
                
    return rmatrix       

In [131]:
def extend_matrix_2(matrix, scale):
    rows = len(matrix)
    cols = len(matrix[0])

    rrows = rows * scale
    rcols = cols * scale

    rmatrix = [[0 for _ in range(rcols)] for _ in range(rrows)]

    # bilinear interpolation

    # vertical
    for y in range(0, rrows, scale):
        for x in range(0, rcols, scale):
                rmatrix[x][y] = matrix[x // scale][y // scale]
      

    for y in range(0, rrows, scale):
        for x in range(rcols - scale):
            if x % scale != 0:
                x1 = x - (x % scale)
                x2 = x1 + scale
                            
                rmatrix[x][y] = ( (x2 - x)*rmatrix[x1][y] + (x - x1)*rmatrix[x2][y] ) / scale
    
    
    for y in range(0, rrows, scale):
        for x in range(rcols - scale, rcols):
            rmatrix[x][y] = rmatrix[rcols - scale][y]

    # horizontal
    for y in range(rrows - scale):
        for x in range(rcols):
            if y % scale != 0:
                y1 = y - (y % scale)
                y2 = y1 + scale
                            
                rmatrix[x][y] = ( (y2 - y)*rmatrix[x][y1] + (y - y1)*rmatrix[x][y2] ) / scale
    
    for y in range(rrows - scale, rrows):
        for x in range(rcols):
            rmatrix[x][y] = rmatrix[x][rrows - scale]
                     
                
    return rmatrix     

In [132]:
def mse(matrix, rmatrix, scale, option):
    N = len(rmatrix)
    M = len(rmatrix[0])
    sum = 0
    for i in range(N):
        for j in range(M):
            if option == '1':       
                sum += (rmatrix[i][j] - matrix[i * scale][j * scale]) ** 2
            elif option == '2':
                sum += (rmatrix[i][j] - matrix[i // scale][j // scale]) ** 2

    return sum / N*M

In [135]:
image = im.open("barbara.ascii.pgm")
#image.show()

matrix = pgm_to_matrix(image)
#print_arr(matrix)

rmatrix = []

option = str(input("Enter the option: "))
scale = int(input("Enter the scale: "))

if option == '1':
    rmatrix = reduce_matrix(matrix, scale)
elif option == '2':
    rmatrix = extend_matrix_2(matrix, scale) # można zmienić na extend_matrix_1, ale 2 jest lepsza.
else:
    print("No option!")
    exit()
    
#print_arr(rmatrix)
print("MSE: ", mse(matrix, rmatrix, scale, option))

image = matrix_to_pgm(rmatrix)
image.show()




MSE:  462685278.2199309
